##### Common Alpha Factors

In [1]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd
import pandas_datareader.data as web

from pathlib import Path
import numpy as np
import pandas as pd
import gc

import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.preprocessing import scale
import talib

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style('whitegrid')
idx = pd.IndexSlice
deciles = np.arange(.1, 1, .1).round(1)

##### Load Data

In [4]:
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')

lock_path = "/tmp/assets_h5_file.lock"  # Choose a path for the lock file

top = 250

from filelock import FileLock

with FileLock(lock_path):
    with pd.HDFStore(DATA_STORE) as store:
        data = store[f'data/top{top}_dataset']


In [5]:
# # with pd.HDFStore(DATA_STORE) as store:
# #     store.put('factors/common', df_optimized)
# data.rename(columns={'market cap': \
#     'market_cap'}, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 671463 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2023-08-11 00:00:00'))
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   open                671463 non-null  float32
 1   high                671463 non-null  float32
 2   low                 671463 non-null  float32
 3   close               671463 non-null  float32
 4   volume              671463 non-null  float64
 5   market_cap          671463 non-null  float64
 6   sector              671463 non-null  float32
 7   ret_frac_order      671463 non-null  float32
 8   ret_01d             671463 non-null  float32
 9   ret_02d             671463 non-null  float32
 10  ret_03d             671463 non-null  float32
 11  ret_04d             671463 non-null  float32
 12  ret_05d             671463 non-null  float32
 13  ret_10d             671463 non-null  float32
 14  ret_21d   

##### TA-Lib: Function Groups

List number of available functions by group

In [6]:
function_groups = ['Overlap Studies',
                   'Momentum Indicators',
                   'Volume Indicators',
                   'Volatility Indicators',
                   'Price Transform',
                   'Cycle Indicators',
                   'Pattern Recognition',
                   'Statistic Functions',
                   'Math Transform',
                   'Math Operators']

talib_grps = talib.get_function_groups()

In [7]:
import pandas as pd
import talib.abstract as ta
import logging
import talib
import numpy as np

# The 5 days can be seen as a very short-term trend.
# The 21 days can be used to determine the medium-term trend.
# The 63 days can be viewed as a longer-term trend.

def compute_talib_indicators(df, function_groups, timeperiods=[5, 21, 63]):
    """
    Compute indicators for the specified function groups using TA-Lib's Abstract API.
    
    Parameters:
    - df: DataFrame with columns 'open', 'high', 'low', 'close', and optionally 'volume'.
    - function_groups: List of function groups to compute.
    """
    
    # Organize data in the format needed for TA-Lib Abstract API
    inputs = {
        'open': df['open'].astype(float).values,
        'high': df['high'].astype(float).values,
        'low': df['low'].astype(float).values,
        'close': df['close'].astype(float).values,
        'volume': df['volume'].astype(float).values if \
            'volume' in df.columns else np.random.random(len(df))
    }

    talib_functions = {k: v for k, v in talib.get_function_groups().items() if k in function_groups}

    success_count = 0
    failure_count = 0
    
    for group, indicators in talib_functions.items():
        for indicator_name in indicators:
            for timeperiod in timeperiods:
                try:
                    indicator_func = ta.Function(indicator_name)

                    parameters = {}
                    if 'timeperiod' in indicator_func.parameters:
                        parameters['timeperiod'] = timeperiod

                    if indicator_name == 'MAVP':
                        inputs['periods'] = np.full(len(df), timeperiod, dtype=np.float64)

                    outputs = indicator_func(inputs, **parameters)

                    # For multi-output functions, like 'BBANDS'
                    if isinstance(outputs, (list, tuple)):
                        for i, out_name in enumerate(indicator_func.output_names):
                            col_name = f"{out_name}_{timeperiod}".upper()
                            df[col_name] = outputs[i]
                    else:
                        col_name = f"{indicator_name}_{timeperiod}".upper()
                        df[col_name] = outputs

                    success_count += 1
                except Exception as e:
                    logging.warning(f"{indicator_name}_{timeperiod} failed due to: {str(e)}")
                    failure_count += 1

    print(f"{success_count} indicators added successfully.")
    print(f"{failure_count} indicators failed.")

    return df

function_groups = [
    'Overlap Studies',
    'Momentum Indicators',
    'Volume Indicators',
    'Volatility Indicators',
    'Price Transform',
    'Cycle Indicators',
    'Pattern Recognition',
    'Statistic Functions',
    'Math Transform',
    'Math Operators'
]

# Assuming you've read your dataframe into a variable named 'data'
data_ta = compute_talib_indicators(data.copy(), function_groups)

474 indicators added successfully.
0 indicators failed.


In [8]:
data_ta.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 671463 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2023-08-11 00:00:00'))
Columns: 526 entries, open to OBV_63
dtypes: float32(29), float64(299), int32(198)
memory usage: 2.1+ GB


In [9]:
from utils import optimize_dataframe
data_ta = optimize_dataframe(data_ta.copy())
data_ta.to_hdf(DATA_STORE, f'factor/top{top}_dataset_with_TA', \
    format='table', mode='a')

Data memory before optimization: 2115.85 MB
Data memory after optimization: 1414.01 MB
Reduced by: 33.17%


In [10]:
data_ta.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 671463 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2023-08-11 00:00:00'))
Columns: 520 entries, open to OBV_63
dtypes: float32(291), float64(31), int32(198)
memory usage: 1.4 GB


In [11]:
data_ta

open        high         low       close  \
ticker date                                                         
AA     2013-01-03   18.904100   19.306499   18.883200   19.137600   
       2013-01-04   19.222000   19.579800   19.200300   19.537100   
       2013-01-07   19.537100   19.600599   19.200300   19.200300   
       2013-01-08   19.327400   19.389099   19.158400   19.200300   
       2013-01-09   19.685101   19.685101   19.115700   19.158400   
...                       ...         ...         ...         ...   
ZTS    2023-08-07  180.190002  181.880005  180.029999  180.690002   
       2023-08-08  182.149994  190.539993  177.250000  189.300003   
       2023-08-09  189.940002  190.949997  186.964996  189.350006   
       2023-08-10  190.000000  191.824997  187.729996  189.100006   
       2023-08-11  188.479996  191.600006  188.179993  190.830002   

                         volume    market_cap  sector  ret_frac_order  \
ticker date                                                             
AA     2013-01-03  1.057118e+07  5.703249e+09     4.0        8.914233   
       2013-01-04  8.690739e+06  5.703249e+09     4.0        8.914233   
       2013-01-07  1.031992e+07  5.703249e+09     4.0        8.914233   
       2013-01-08  1.609879e+07  5.703249e+09     4.0        8.914233   
       2013-01-09  1.979932e+07  5.703249e+09     4.0        8.914233   
...                         ...           ...     ...             ...   
ZTS    2023-08-07  1.941699e+06  8.818486e+10     2.0       16.669207   
       2023-08-08  4.117986e+06  8.818486e+10     2.0       24.922548   
       2023-08-09  2.270030e+06  8.818486e+10     2.0       21.726665   
       2023-08-10  2.348639e+06  8.818486e+10     2.0       20.455498   
       2023-08-11  1.742931e+06  8.818486e+10     2.0       21.730034   

                    ret_01d   ret_02d   ret_03d   ret_04d   ret_05d   ret_10d  \
ticker date                                                                     
AA     2013-01-03  0.020875  0.003276  0.003276  0.001087 -0.011062 -0.014385   
       2013-01-04  0.020875  0.003276  0.003276  0.001087 -0.011062 -0.014385   
       2013-01-07 -0.017239  0.003276  0.003276  0.001087 -0.011062 -0.014385   
       2013-01-08  0.000000 -0.017239  0.003276  0.001087 -0.011062 -0.014385   
       2013-01-09 -0.002182 -0.002182 -0.019384  0.001087 -0.011062 -0.014385   
...                     ...       ...       ...       ...       ...       ...   
ZTS    2023-08-07  0.011532 -0.001050 -0.007198 -0.025982 -0.039343 -0.026350   
       2023-08-08  0.047651  0.059732  0.046550  0.040110  0.020430 -0.004470   
       2023-08-09  0.000264  0.047927  0.060012  0.046827  0.040385 -0.013802   
       2023-08-10 -0.001320 -0.001057  0.046544  0.058613  0.045444  0.002173   
       2023-08-11  0.009149  0.007816  0.008082  0.056118  0.068298  0.004897   

                    ret_21d   ret_42d   ret_63d  ret_126d  ret_252d  \
ticker date                                                           
AA     2013-01-03 -0.023211 -0.051981 -0.088433 -0.133089  0.181219   
       2013-01-04 -0.023211 -0.051981 -0.088433 -0.133089  0.181219   
       2013-01-07 -0.023211 -0.051981 -0.088433 -0.133089  0.181219   
       2013-01-08 -0.023211 -0.051981 -0.088433 -0.133089  0.181219   
       2013-01-09 -0.023211 -0.051981 -0.088433 -0.133089  0.181219   
...                     ...       ...       ...       ...       ...   
ZTS    2023-08-07  0.081070  0.062008 -0.029748  0.076818  0.038806   
       2023-08-08  0.112090  0.157232  0.023963  0.147760  0.086433   
       2023-08-09  0.112907  0.152465  0.027011  0.143625  0.090468   
       2023-08-10  0.100314  0.159482  0.021389  0.166780  0.117672   
       2023-08-11  0.115965  0.152703  0.030845  0.186680  0.098675   

                   ret_fwd_frac_order  ret_fwd_01d  ret_fwd_02d  ret_fwd_03d  \
ticker date                                                                    
AA     2013-01-03            8.914233  

In [12]:
del data_ta

##### Rolling Factor Betas

In [13]:
factor_data = (web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', 
                              start=2005)[0].rename(columns={'Mkt-RF': 'MARKET'}))
factor_data.index.names = ['date']

In [14]:
factors = factor_data.columns[:-1]
factors

Index(['MARKET', 'SMB', 'HML', 'RMW', 'CMA'], dtype='object')

In [15]:
t = 1
# ret = f'ret_{t:02}d'
ret = 'ret_frac_order'
windows = [21, 63, 252]
for window in windows:
    print(window)
    betas = []
    for ticker, df in data.groupby('ticker', group_keys=False):
        model_data = df[[ret]].merge(factor_data, on='date').dropna()
        model_data[ret] -= model_data.RF

        rolling_ols = RollingOLS(endog=model_data[ret], 
                                 exog=sm.add_constant(model_data[factors]), window=window)
        factor_model = rolling_ols.fit(params_only=True).params.rename(columns={'const':'ALPHA'})
        result = factor_model.assign(ticker=ticker).set_index('ticker', append=True).swaplevel()
        betas.append(result)
    betas = pd.concat(betas).rename(columns=lambda x: f'{x}_{window:02}')
    data = data.join(betas)

21
63
252


##### Size proxy

In [16]:
by_ticker = data.groupby('ticker', group_keys=False)
data['size_factor'] = by_ticker.close.apply(lambda x: x.fillna(method='bfill').div(x.iloc[0]))
data['size_proxy'] = data['market_cap'].mul(data.size_factor).div(1e6)

In [17]:
from utils import optimize_dataframe
data = optimize_dataframe(data.copy())
data.to_hdf(DATA_STORE, \
    f'factor/top{top}_dataset_with_rolling_beta_size_proxy', \
        format='table', mode='a')

Data memory before optimization: 1140.39 MB
Data memory after optimization: 859.23 MB
Reduced by: 24.66%


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4094783 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2023-08-11 00:00:00'))
Data columns (total 51 columns):
 #   Column              Dtype  
---  ------              -----  
 0   open                float32
 1   high                float32
 2   low                 float32
 3   close               float32
 4   volume              float64
 5   market_cap          float64
 6   sector              float32
 7   ret_frac_order      float32
 8   ret_01d             float32
 9   ret_02d             float32
 10  ret_03d             float32
 11  ret_04d             float32
 12  ret_05d             float32
 13  ret_10d             float32
 14  ret_21d             float32
 15  ret_42d             float32
 16  ret_63d             float32
 17  ret_126d            float32
 18  ret_252d            float32
 19  ret_fwd_frac_order  float32
 20  ret_fwd_01d         float32
 21  ret_fwd_02d         float32
 22  ret_fwd_03d      

In [19]:
del data

In [20]:
from utils import clear_large_vars
clear_large_vars(threshold_size_in_MB=100)